In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Concatenate
from tensorflow.keras.models import Sequential
import numpy as np


class reorg_layer(tf.keras.layers.Layer):
    def __init__(self, stride):
        super(reorg_layer, self).__init__()
        self.stride = stride
    def call(self, input):
        x = tf.image.extract_patches(input,
                                     sizes=[1, self.stride, self.stride, 1],
                                     strides=[1, self.stride, self.stride, 1],
                                     rates=[1, 1, 1, 1],
                                     padding='VALID')
        return x

def get_UnifiedNet(input_shape):
    input = tf.keras.Input(shape = input_shape)

    x = Conv2D(32, (3,3), padding = 'same', input_shape = (416, 416, 3))(input)
    x = MaxPool2D(strides = (2, 2), padding = 'same')(x)

    x = Conv2D(64, (3,3), padding = 'same')(x)
    x = MaxPool2D(strides = (2, 2), padding = 'same')(x)

    x = Conv2D(128, (3,3), padding = 'same')(x)
    x = Conv2D(64, (1,1), padding = 'same')(x)
    x = Conv2D(128, (3,3), padding = 'same')(x)
    x = MaxPool2D(strides = (2, 2), padding = 'same')(x)

    x = Conv2D(256, (3,3), padding = 'same')(x)
    x = Conv2D(128, (1,1), padding = 'same')(x)
    x = Conv2D(256, (3,3), padding = 'same')(x)
    x = MaxPool2D(strides = (2, 2), padding = 'same')(x)

    x = Conv2D(512, (3,3), padding = 'same')(x)
    x = Conv2D(256, (1,1), padding = 'same')(x)
    x = Conv2D(512, (3,3), padding = 'same')(x)
    x = Conv2D(256, (1,1), padding = 'same')(x)
    route1 = Conv2D(512, (3,3), padding = 'same')(x)

    x = MaxPool2D(strides = (2, 2), padding = 'same')(route1)
    x = Conv2D(1024, (3,3), padding = 'same')(x)
    x = Conv2D(512, (1,1), padding = 'same')(x)
    x = Conv2D(1024, (3,3), padding = 'same')(x)
    x = Conv2D(512, (1,1), padding = 'same')(x)
    x = Conv2D(1024, (3,3), padding = 'same')(x)
    x = Conv2D(1024, (3,3), padding = 'same')(x)
    route2 = Conv2D(1024, (3,3), padding = 'same')(x)

    x = Conv2D(64, (1,1), padding = 'same')(route1)
    x = reorg_layer(2)(x)
    x = Concatenate()([x, route2])

    x = Conv2D(1024, (3,3), padding = 'same')(x)
    x = Conv2D(720, (1,1), padding = 'same')(x)
    model = tf.keras.Model(inputs = [input],
                           outputs = [x],
                           name = 'UnifiedNet')
    return model

In [ ]:
un = get_UnifiedNet((416, 416, 3))
un.summary()

Model: "UnifiedNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 416, 416, 3) 0                                            
__________________________________________________________________________________________________
conv2d_75 (Conv2D)              (None, 416, 416, 32) 896         input_8[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_17 (MaxPooling2D) (None, 208, 208, 32) 0           conv2d_75[0][0]                  
__________________________________________________________________________________________________
conv2d_76 (Conv2D)              (None, 208, 208, 64) 18496       max_pooling2d_17[0][0]           
_________________________________________________________________________________________